author:  Dr Milto Miltiadou
version: 1.0


In [1]:

import sys

# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'ee'
if modulename not in sys.modules: 
   # import GEE and Authenticate, token or log in will be asked from web browser
   import ee
   #ee.Authenticate()
   ee.Initialize()
else:
   print('GEE already imported')
   # google earth engine already imported and authenticated

import pandas as pd
import numbers
from datetime import date
import os


In [ ]:


class FieldData: 
    # constructor
    def __init__(self,properties):
        self.proj = None
        self.csvfile = None
        self.colX = None
        self.colY = None
        self.radius = None
        self.lengthX = None
        self.lengthY = None
        self.plotcode = None
        self.filedataFilewithIDs = None
        self.shp = None
        self.bufferredPoints = None
        self.df = None
        
        if ("shpfilename" in properties):
            self.shp = properties["shpfilename"]
        
        if ('outPlotFileWithIDs'in properties):
            self.filedataFilewithIDs = properties['outPlotFileWithIDs']
        else:
            raise Exception ("ERROR: name of file with plots and ids to be exported not defined!")

  
        if ('csvfilename' in properties):
            self.csvfile = properties['csvfilename']
        #else:
        #    raise Exception("CSV input file should be defined as follow  '\csvfilename'\:<csvfilename>")
            
            # if csvfile defined then load the rest relevant parameters
            if ('proj' in properties):        
                self.proj = ee.Projection(properties['proj'])
            else:
                print('WARNING: projection was not defined. Default project is ', self.proj)

            if ('radius' in properties):
                radius = properties['radius']
                if(isinstance(radius,numbers.Number) and radius>0):
                    self.radius=radius
                else:
                    print("WARNING: Wrong radius provided : ", radius)
            #else: 
                # Maybe it is a rectangular plot - further testing later
                
            if ('lengthX' in properties):
                lengthX = properties['lengthX']
                if(isinstance(lengthX,numbers.Number) and lengthX>0):
                    self.lengthX = lengthX
                else:
                    print("WARNING: Wrong lengthX provided : ", lengthX)
            # else:
                # Maybe it is a circular plot - further testing later
            
            if ('lengthY' in properties):
                lengthY = properties['lengthY']
                if(isinstance(lengthY,numbers.Number) and lengthY>0):
                    self.lengthY = lengthY
                else:
                    print("WARNING: Wrong lengthY provided : ", lengthY)
            # else:
                # Maybe it is a circular plot - further testing later

            if('xcol' in properties):
                self.colX = properties['xcol']
            else :
                print ("WARNING: name of column containing X coordinates was not provided. Set to \"CX\"")
                self.colX = "CX"
            if('ycol' in properties):
                self.colY = properties['ycol']
            else :
                print ("WARNING: name of column containing Y coordinates was not provided. Set to \"CY\"")
                self.colX = "CY"
            
        if (self.csvfile is None and self.shp is None):
            raise Exception("Either a csv file or a shapefile should be loaded")

        if(self.shp is not None and isinstance(self.shp,str)):
            print (self.shp, " has been loaded")
        elif (self.csvfile is not None and isinstance(self.csvfile, str)) : 
            print (self.csvfile, "has been loaded")
        else :
            raise Exception("Either a csv file or a shapefile should be loaded")

        if (self.csvfile is not None):
            if (self.proj is None or self.csvfile is None or self.colX is None or self.colY is None):
                raise Exception ("Field data defining CSV file: Variables not defined correctly")
        
            if not(self.radius!=None or (self.lengthX!=None and self.lengthY!=None)):                           
                raise Exception("Either radius for circular plots or lengthX and legthY for rectangular plots need",
                            " to be defined within the field data.")
            
            low_memory=False
            self.df = pd.read_csv(self.csvfile, low_memory=False) 
            # Removing Nan values
            self.df.dropna(subset=[self.colX],inplace=True)
            self.df.dropna(subset=[self.colY],inplace=True)
            self.df = self.df.reset_index()
            self.indexLb = "indexField"
            self.df[self.indexLb] = list(range(1,len(self.df)+1))
       
        
            ### TO DO ~~~ export identifiers for shapefiles
            directory = os.path.dirname(self.filedataFilewithIDs)
            if directory and not os.path.exists(directory):
                os.makedirs(directory)
            # else directory and subdirectories exist
            self.exportPlotDataWithAddedIdentifiers(self.filedataFilewithIDs)
            
        self.sampleSize = 300

    
    # used in polygons to define buffer points as they are differently created
    def setBufferPoints(self, polygonsCollection):
        self.bufferedPoints = polygonsCollection
        
    
    # method that returns the number of rows/plot data contained within the csv file
    def getLen(self):
        if (self.shp is None and self.csvfile is not None):
            return len(self.df.index)
        elif (self.shp is not None):
            self.bufferredPoints = ee.FeatureCollection(self.shp)
            feature_count = self.bufferredPoints.size().getInfo()
            return feature_count
        else : 
            raise Exception ("Possibly both shp and csv are loading producing confusion")
   
    """
    # 2nd constructor that can reset the class using a given dataframe
    def reset(self, dataframe, proj):
        self.proj = proj 
        low_memory=False
        self.df = dataframe
        self.distance = 100
        if self.indexLb in self.df.columns:
           self.df.drop(columns=[self.indexLb])
        self.df[self.indexLb] = list(range(1,len(self.df)+1))
        self.bufferredPoints = None
        self.sampleSize = 300
        self.exportPlotDataWithAddedIdentifiers(self.filedataFilewithIDs)
    """

    ## method that returns the smallest and bigger available year within the field data
    ## @param[in] yearCol the name of the column that states the years
    ## @returns [min,max] the smallest and bigger year included
    def getMinMaxYear(self,yearlabel):
        years = self.df[yearlabel]
        return([min(years),max(years)])
    
    # @brief method that keeps the years of interests (inclusive) and discards the rest
    def filterYearsOfInterest(self,startYear,endYear,yearlabel):
        self.df = self.df[self.df[yearlabel].isin(list(range(startYear,endYear+1)))]
        self.df = self.df.reset_index()
        return None
    
    def bufferPoint(self,feature):
        return feature.buffer(self.distance, 1)
    
    ## method that returns a dataframe containing the data of the year of interest
    #  @param[in] year the year of interest
    #  @param[in] yearlabel the name of the column containing the years
    def getYearOfInterest(self,year,yearlabel):        
        tmpdf = self.df[self.df[yearlabel] == year]
        tmpdf = tmpdf.reset_index()
        return tmpdf
    
    ## NOT WORKING
    def createCircularBufferPoints(self,currentMin,currentMax):
        print("Creating Circular plots")
        if (len(self.df)==0):
            # then dataframe has no rows
            return None 
        tmpdf = self.df.iloc[currentMin:currentMax]

        if tmpdf.empty:
            return None
       # create a feature collection with the first location
        x = 0
        y = 0 
        indx = 0
        for index, row in tmpdf.iterrows():
            x    = row[self.colX]
            y    = row[self.colY]
            indx = row[self.indexLb]
            break

        self.bufferredPoints = None
        # pointList is not defined in a loop to make sure memory allocation is preserved after 
        # the loop is deleted
        self.bufferredPoints = ee.FeatureCollection(
              [ee.Feature(
                 ee.Geometry.Point(
                        [x,y],self.proj
                        ),
                    {
                        self.indexLb : indx,
                        "system:index" : "0"
                    }
                    ).buffer(self.radius)]
                )
        
        # add other locations to the feature collection
        i = 0
        for index, row in tmpdf.iterrows():
            if i==0 :
                i = 1
                continue
            self.bufferredPoints = self.bufferredPoints.merge(ee.FeatureCollection(
                [ee.Feature(
                    ee.Geometry.Point(
                        [row[self.colX],row[self.colY]],self.proj
                        ),
                    {
                        self.indexLb : row[self.indexLb],
                        "system:index" : str(i)
                    }
                    ).buffer(self.radius)]
                ))
        
        return self.bufferredPoints
    
    def createRectangularPlots(self, currentMin, currentMax):
        print("Creating Rectangular plots")
        if len(self.df) == 0:
            # the dataframe has no rows
            return None

        tmpdf = self.df.iloc[currentMin:currentMax]

        if tmpdf.empty:
            return None

        # create a feature collection with the first location
        x = 0
        y = 0
        indx = 0
        for index, row in tmpdf.iterrows():
            x = row[self.colX]
            y = row[self.colY]
            indx = row[self.indexLb]
            break

        self.bufferedPoints = None
        # pointList is not defined in a loop to make sure memory allocation is preserved after
        # the loop is deleted

        self.bufferedPoints = ee.FeatureCollection([
            ee.Feature(
                ee.Geometry.Polygon(
                    coords=[
                        [x, y],
                        [x, y + self.lengthY],
                        [x + self.lengthX, y + self.lengthY],
                        [x + self.lengthX, y]
                    ],
                    geodesic=True,
                    proj=self.proj                    
                ),
                {
                    self.plotcode: row[self.plotcode],
                    self.indexLb: indx,
                    "system:index": "0"
                }
            )
        ])
        
        print(x, y)
        print(x, y + self.lengthY)
        print(x + self.lengthX, y + self.lengthY)
        print(x + self.lengthX, y)
        print("***********************")
        
        geometry = ee.Geometry.Polygon(
           coords=[[x, y], [x, y + self.lengthY], [x + self.lengthX, y + self.lengthY], [x + self.lengthX, y]],
            proj=self.proj
        )
        #print("Polygon:", geometry)

        # add other locations to the feature collection
        for i, (index, row) in enumerate(tmpdf.iterrows()):
            if i == 0:
                continue
            x = row[self.colX]
            y = row[self.colY]
            indx = row[self.indexLb]
            print(x, y)
            print(x, y + self.lengthY)
            print(x + self.lengthX, y + self.lengthY)
            print(x + self.lengthX, y)
            print("***********************")
            self.bufferedPoints = self.bufferedPoints.merge(ee.FeatureCollection([
                ee.Feature(
                    ee.Geometry.Polygon(
                        coords=[
                            [x, y],
                            [x, y + self.lengthY],
                            [x + self.lengthX, y + self.lengthY],
                            [x + self.lengthX, y]
                        ],
                        geodesic=True ,
                        proj=self.proj
                    ),
                    {
                        self.plotcode: row[self.plotcode],
                        self.indexLb: indx,
                        "system:index": str(i)
                    }
                )
            ]))

        return self.bufferedPoints
        
    """
    def createRectangularPlots(self,currentMin,currentMax):
        print("Creating Rectangular plots")
        if (len(self.df)==0):
            # then dataframe has no rows
            return None 
        tmpdf = self.df.iloc[currentMin:currentMax]

        if tmpdf.empty:
            return None
       # create a feature collection with the first location
        x = 0
        y = 0 
        indx = 0
        for index, row in tmpdf.iterrows():
            x    = row[self.colX]
            y    = row[self.colY]
            indx = row[self.indexLb]
            break

        self.bufferredPoints = None
        # pointList is not defined in a loop to make sure memory allocation is preserved after 
        # the loop is deleted
        
        self.bufferredPoints  = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[x             ,y             ],
                 [x             ,y+self.lengthY],
                 [x+self.lengthX,y+self.lengthY],
                 [x+self.lengthX,y             ]],proj=self.proj),
            {
               self.indexLb : indx,
               "system:index": "0"
            })])
        
        
        
        # add other locations to the feature collection
        i = 0
        for index, row in tmpdf.iterrows():
            if i==0 :
                i = 1
                continue
            x    = row[self.colX]
            y    = row[self.colY]
            indx = row[self.indexLb]
            self.bufferredPoints = self.bufferredPoints.merge( ee.FeatureCollection(
             [ee.Feature(
               ee.Geometry.Polygon(
                [[x             ,y             ],
                 [x             ,y+self.lengthY],
                 [x+self.lengthX,y+self.lengthY],
                 [x+self.lengthX,y             ]],proj=self.proj),
                 {
                    self.indexLb : indx,
                    "system:index":str(i) 
                })]))
            
        
        return self.bufferredPoints
    """
    ## method that reads the centres of currentMin to currentMax plots and creates a list of circular polygons with 
    #  centres the centre of each plot and radius the predefined radius of the plots
    # @param[in] currentMin used to load a sample of the data this is the min value of the field range to be loaded
    # @param[in] currentMax used to load a sample of the data this is the max value of the field date range to be loaded
    def createBufferedPoints(self,currentMin,currentMax):
        if (self.radius!=None):
            return self.createCircularBufferPoints(currentMin,currentMax) 
        elif (self.lengthX!=None and self.lengthY!=None):
            return self.createRectangularPlots(currentMin,currentMax)

    
    def exportfeatureVectorsToDrive(self,collection, outCsvFeatureVectors, driveFolder, iscale):
        if (self.pointsList == None) : 
            print ("Plot data have not been read yet. Please call \"createBufferedPoints", \
            "(currentMin,currentMax)\" first")
            return
   
        training = collection.sampleRegions(
            collection = self.pointsList,
            properties = [self.indexLb],
            scale      = iscale,
            projection = self.proj,
            geometries = True
        )
        
        # TO DO: COMMENT WHEN NOT TESTING OUTPUT AS BATCH COMMANDS ARE LIMITED
        print("STARTING BATCH SCRIPT FOR EXPORTING FILE")
        task = ee.batch.Export.table.toDrive(**{
            'collection' : training,
            'description' : outCsvFeatureVectors,
            'folder' : driveFolder,
            'fileFormat' : "CSV"
        })
        #task.start()
        print("END OF CALLING BATCH SCRIPT")   

  


    def exportPlotDataWithAddedIdentifiers(self, nameOfNewPlotFile):
        #print("exporting field data to ", nameOfNewPlotFile)
        self.df.to_csv(nameOfNewPlotFile)

    def printFieldData(self):
        print(self.df.to_string())  



    # get mean and std for one band of an image for each buffered point
    def getInfoRegions(self,collection,bandName, bpoints):
    # bnamestr = bandName.get('band')
        return collection.select([bandName]).reduceRegions(**{
            'collection': bpoints.select("indexField"),
            'reducer': ee.Reducer.mean().combine(**{
                'reducer2': ee.Reducer.stdDev(),
                'sharedInputs': True
            }),
            'scale': 10#,
            #'bestEffort': True  # Use maxPixels if you care about scale.
        }).map(lambda feature: feature.set('bandName',bandName))  \
        .filter(ee.Filter.neq('mean',None))


    def getFeatureCollection(self,collection,bpoints):
        bandNamesImg = collection.bandNames().getInfo()
        print('Band names: ', bandNamesImg)
        for band in bandNamesImg :
            if(not isinstance(band,str)):
                bandNamesImg.remove(band)
        featureCollection1 = ee.FeatureCollection([])
        for band in bandNamesImg:
            features = self.getInfoRegions(collection,band,bpoints)
            featureCollection1 = featureCollection1.merge(features)        
        return featureCollection1





    def processMatchesMean(self,row):
        # Get the list of all features with a unique row ID.
        matches = ee.List(row.get('matches'))
        # Map a function over the list of rows to return a list of column ID and value.
        values = matches.map(lambda feature: [ee.Feature(feature).get('bandName'), ee.Feature(feature).get('mean')])
        # Return the row with its ID property and properties for all matching column IDs storing the output of the reducer.
        return row.select(['indexField']).set(ee.Dictionary(values.flatten()))

    ## Format a table of triplets into a 2D table of rowId x colId.
    def formatMean (self,table):
        # Get a FeatureCollection with unique row IDs.
        rows = table.distinct('indexField')
        filterEq = ee.Filter.equals(leftField='indexField', rightField='indexField')
        innerJoin = ee.Join.saveAll('matches')
        toyJoin = innerJoin.apply(primary=rows, secondary=table, condition=filterEq)
        
        return toyJoin.map(algorithm = self.processMatchesMean)

    def processMatchesStd(self,row):
        # Get the list of all features with a unique row ID.
        matches = ee.List(row.get('matches'))
        # Map a function over the list of rows to return a list of column ID and value.
        values = matches.map(lambda feature: [ee.Feature(feature).get('bandName'),
                                              ee.Feature(feature).get('stdDev')])
             
        # Return the row with its ID property and properties for all matching column IDs storing the output of the reducer.
        return row.select(['indexField']).set(ee.Dictionary(values.flatten()))


    ## Format a table of triplets into a 2D table of rowId x colId.
    def formatStd (self,table):
        # Get a FeatureCollection with unique row IDs.
        rows = table.distinct('indexField')
        filterEq = ee.Filter.equals(leftField='indexField', rightField='indexField')
        innerJoin = ee.Join.saveAll('matches')
        toyJoin = innerJoin.apply(primary=rows, secondary=table, condition=filterEq)
        return toyJoin.map(algorithm = self.processMatchesStd)        
        
    
    # collection = s2bands
    # there is a copy of this function in PlotToSat class for shapefiles
    def exportFeaturesMeanStdCSV(self,collection,ouutCsvFeatureVectors,driveFolder):
        if (self.bufferredPoints == None):
            raise Exception("Please call createBufferedPoints(currentMin,currentMax) function first" )
        featureCollection = self.getFeatureCollection(collection,self.bufferredPoints)

        tableMean = self.formatMean(featureCollection)
        tableStd  = self.formatStd (featureCollection)
               
        meanName = ouutCsvFeatureVectors+"_mean"
        stdName = ouutCsvFeatureVectors+"_stdD"
        print ("START EXPORTING FEATURES VECTORS OF A SINGLE FILE")
        task = ee.batch.Export.table.toDrive(**{
            'collection':tableMean,
            'description':meanName,
            'folder': driveFolder,
            'fileFormat':'CSV'
        })
        task.start()

        task = ee.batch.Export.table.toDrive(**{
            'collection':tableStd,
            'description':stdName,
            'folder': driveFolder,
            'fileFormat':'CSV'
        })
        task.start()


In [ ]:
print ("Class fieldData imported")

Personal Notes:

# coordinate system of Spain field data: EPSG: 3042

# info from: https://gis.stackexchange.com/questions/362582/coordinate-system-mismatch-in-folium 
# all coordinates passed to Leaflet functions/methods are always EPSG:4326.
# [latitude, longitude]

# when defining/creating map with folium.Map call, actual crs of map has to be specified. Map is in EPSG:3857

# transforming coordinates with gdal:
# https://gdal.org/programs/gdaltransform.html
